
1. Pyspark :Joining DataFrames in PySpark Without Duplicate Columns 
2. sql: find out the customer who consecutive ordered particular product within 3odays of first one bought date
3. leftanti join synatx.(scenario, identify which join and write syntax) 
4. create a datafrmae with below, where incentive is string. Add new column with the resultant of salary*incentive(atfer converting 'incentive'  to Int)

dept     salary  incentive

Sales     3000   10.00

Sales     4600   10.02

Sales     4100   10.15

6. Window function usage, use pyspark dataframe below, find the second rank of salary of each dept  (window function , second lowest _salary for each dept )

dept     salary

Sales     3000

Sales     4600

Sales     4100

df.groupBy('dept').

7. How to debug pyspark 


* **Question.1: Window function usage, use pyspark dataframe below, find the second rank of salary of each dept  (window function , second lowest _salary for each dept )**

In [0]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('test').getOrCreate()

In [0]:
employees_Salary = [("James", "Sales", 2000,'10'),("sofy", "Sales", 3000,'10'),("Laren", "Sales", 4000, '10'),("Kiku", "Sales", 5000, '10'),("Sam", "Finance", 6000, '10'),("Samuel", "Finance", 7000, '10'),("Yash", "Finance", 8000, '10'),("Rabin", "Finance", 9000, '10'),("Lukasz", "Marketing", 10000, '10'),("Jolly", "Marketing", 11000, '10'),("Mausam", "Marketing", 12000, '10'),("Lamba", "Marketing", 13000, '10'),("Jogesh", "HR", 14000, '10'),("Mannu", "HR", 15000, '10'),("Sylvia", "HR", 16000, '10'),("Sama", "HR", 17000, '10'),
]

In [0]:
df=spark.createDataFrame(employees_Salary,schema=['Name', 'Dept','Salary', 'incentives'])
# df.show()
# df.printSchema()

print(type(df))
df.createOrReplaceTempView('emp')

spark.sql("""
          with CTE_rank as (
              select * , dense_rank() over(partition by Dept  order by salary) as rr from emp
          )

          select * from cte_rank where rr = 2       
          """).show()




<class 'pyspark.sql.dataframe.DataFrame'>
+------+---------+------+----------+---+
|  Name|     Dept|Salary|incentives| rr|
+------+---------+------+----------+---+
|Samuel|  Finance|  7000|        10|  2|
| Mannu|       HR| 15000|        10|  2|
| Jolly|Marketing| 11000|        10|  2|
|  sofy|    Sales|  3000|        10|  2|
+------+---------+------+----------+---+



In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank


window=Window.partitionBy('Dept').orderBy('Salary')
dd=df.select(df.Name, df.Dept, df.Salary, dense_rank().over(window).alias('rn'))
dd.filter("rn=2").show()

+------+---------+------+---+
|  Name|     Dept|Salary| rn|
+------+---------+------+---+
|Samuel|  Finance|  7000|  2|
| Mannu|       HR| 15000|  2|
| Jolly|Marketing| 11000|  2|
|  sofy|    Sales|  3000|  2|
+------+---------+------+---+



In [0]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Dept: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- incentives: string (nullable = true)




* **2. Question: new column with the resultant of salary*incentive(atfer converting 'incentive'  to Int)**

In [0]:
df1=df.withColumn('New_incentive_int_type', df.incentives.cast('int'))
df1.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Dept: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- incentives: string (nullable = true)
 |-- New_incentive_int_type: integer (nullable = true)



In [0]:
from pyspark.sql.functions import col, expr
df1.withColumn('new-Salary', col('salary')* col('New_incentive_int_type')).drop('incentives').show()

+------+---------+------+----------------------+----------+
|  Name|     Dept|Salary|New_incentive_int_type|new-Salary|
+------+---------+------+----------------------+----------+
| James|    Sales|  2000|                    10|     20000|
|  sofy|    Sales|  3000|                    10|     30000|
| Laren|    Sales|  4000|                    10|     40000|
|  Kiku|    Sales|  5000|                    10|     50000|
|   Sam|  Finance|  6000|                    10|     60000|
|Samuel|  Finance|  7000|                    10|     70000|
|  Yash|  Finance|  8000|                    10|     80000|
| Rabin|  Finance|  9000|                    10|     90000|
|Lukasz|Marketing| 10000|                    10|    100000|
| Jolly|Marketing| 11000|                    10|    110000|
|Mausam|Marketing| 12000|                    10|    120000|
| Lamba|Marketing| 13000|                    10|    130000|
|Jogesh|       HR| 14000|                    10|    140000|
| Mannu|       HR| 15000|               


* **3.Question:leftanti join synatx.(scenario, identify which join and write syntax)**


In [0]:
records = [(1,"Rahul","2018","10",30000), 
    (2,"Syam","2010","20",40000), 
    (3,"Mohan","2010","10",40000), 
    (4,"Mac","2005","60",35000), 
    (5,"Tom","2010","40",38000)]
record_Columns = ["id","Name","year", "store_id","Cost"]
recordDF = spark.createDataFrame(data=records, schema = record_Columns)
recordDF.show(truncate=False)

store_master = [("Sports",10),("Books",20), ("Women",30), ("Men",40)]
store_master_columns = ["Catagory","Cat_id"]
store_masterDF = spark.createDataFrame(data=store_master, schema = store_master_columns)
store_masterDF.show(truncate=False)

+---+-----+----+--------+-----+
|id |Name |year|store_id|Cost |
+---+-----+----+--------+-----+
|1  |Rahul|2018|10      |30000|
|2  |Syam |2010|20      |40000|
|3  |Mohan|2010|10      |40000|
|4  |Mac  |2005|60      |35000|
|5  |Tom  |2010|40      |38000|
+---+-----+----+--------+-----+

+--------+------+
|Catagory|Cat_id|
+--------+------+
|Sports  |10    |
|Books   |20    |
|Women   |30    |
|Men     |40    |
+--------+------+



In [0]:
recordDF.join(store_masterDF, recordDF.store_id==store_masterDF.Cat_id , how='leftanti').show()

+---+----+----+--------+-----+
| id|Name|year|store_id| Cost|
+---+----+----+--------+-----+
|  4| Mac|2005|      60|35000|
+---+----+----+--------+-----+




* **4.Question:Pyspark :Joining DataFrames in PySpark Without Duplicate Columns**

In [0]:
data = [(('Ram'),1,'M'),
          (('Mike'),2,'M'),
          (('Rohini'),3,'M'),
          (('Maria'),4,'F'),
          (('Jenis'),5,'F')]

columns = ["Name","ID","Gender"]
 
# Create the spark dataframe
df1 = spark.createDataFrame(data=data, schema = columns)
 
# Print the dataframe
df1.show()


# Create data in dataframe
data2 = [(1,3000),
          (2,4000),
          (3,4000),
          (4,4000),
          (5, 1200)]
 
# Column names in dataframe
columns = ["ID","salary"]
 
# Create the spark dataframe
df2 = spark.createDataFrame(data=data2,
                            schema = columns)
 
# Print the dataframe
df2.show()

+------+---+------+
|  Name| ID|Gender|
+------+---+------+
|   Ram|  1|     M|
|  Mike|  2|     M|
|Rohini|  3|     M|
| Maria|  4|     F|
| Jenis|  5|     F|
+------+---+------+

+---+------+
| ID|salary|
+---+------+
|  1|  3000|
|  2|  4000|
|  3|  4000|
|  4|  4000|
|  5|  1200|
+---+------+



In [0]:
#join
df = df1.join(df2, df1.ID==df2.ID)
df.show()  #In result you can see Duplicate column of 'ID'. 

df_noDuplicates = df1.join(df2, ['ID']) #No duplicate column of 'ID'
df_noDuplicates.show()

df_another = df1.join(df2, df1.ID==df2.ID).drop(df1.ID).show()#No duplicate column of 'ID', as we are using 'drop' method

+------+---+------+---+------+
|  Name| ID|Gender| ID|salary|
+------+---+------+---+------+
|   Ram|  1|     M|  1|  3000|
|  Mike|  2|     M|  2|  4000|
|Rohini|  3|     M|  3|  4000|
| Maria|  4|     F|  4|  4000|
| Jenis|  5|     F|  5|  1200|
+------+---+------+---+------+

+---+------+------+------+
| ID|  Name|Gender|salary|
+---+------+------+------+
|  1|   Ram|     M|  3000|
|  2|  Mike|     M|  4000|
|  3|Rohini|     M|  4000|
|  4| Maria|     F|  4000|
|  5| Jenis|     F|  1200|
+---+------+------+------+

+------+------+---+------+
|  Name|Gender| ID|salary|
+------+------+---+------+
|   Ram|     M|  1|  3000|
|  Mike|     M|  2|  4000|
|Rohini|     M|  3|  4000|
| Maria|     F|  4|  4000|
| Jenis|     F|  5|  1200|
+------+------+---+------+



---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-189548932076915>:1
----> 1 df = df1.join(df2, df1.ID==df2.ID).drop(ID).show()

NameError: name 'ID' is not defined